In [21]:
#Importing libraries
import mysql.connector
import numpy as np
import math
import random

In [22]:
#Getting connection to DB
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="imghostrider1",
  database="hundredplus"
)

#printing the DB object 
print(mydb)

In [23]:
#Setting up the cursor
cursor = mydb.cursor()

In [24]:
#Fetching relevant data
cursor.execute("select * from shop_product")
shop_product = cursor.fetchall()

cursor.execute("select * from shop_category_description")
shop_category_description = cursor.fetchall()

cursor.execute("select * from shop_product_description")
shop_product_description = cursor.fetchall()

In [25]:
def remove_duplicates(input_list):
    final_list = [] 
    for num in input_list: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list

In [26]:
#Returning the keywords which will be used to fetch products having similar keywords
def filter_related_keywords(related_keywords):
    related_keywords = related_keywords[0:3]
    return related_keywords

In [27]:
def keyword_related_products(related_keywords):
    products = []
    for keyword in related_keywords:
        query = '''select product_id from shop_product_description where keyword like "%'''+keyword+'''%"'''
        cursor.execute(query)
        products.append(cursor.fetchall())    
    products = products[0]
    products = [item for t in products for item in t] 
    return products

In [28]:
def filter_category_related_products(category_related_products):
    category_related_products = category_related_products[0]
    category_related_products = [item for t in category_related_products for item in t]
    
    return category_related_products

In [29]:
def recommend_products(popular_products, category_related_products, keyword_related_products):
    final_products_to_be_recommended = []
    min_stock = 10
    
    temp_list = popular_products + category_related_products + keyword_related_products
    temp_list = remove_duplicates(temp_list)
    
    query = "select avg(sold) from shop_product;"
    cursor.execute(query)
    avg_sold = cursor.fetchall()
    avg_sold = avg_sold[0]
    avg_sold = avg_sold[0]
    avg_sold = math.ceil(avg_sold)

    query2 = "select avg(view) from shop_product;"
    cursor.execute(query2)
    avg_view = cursor.fetchall()
    avg_view = avg_view[0]
    avg_view = avg_view[0]
    avg_view = math.ceil(avg_view)
    
    iterator = 0
    
    for iterator in range(len(temp_list)):
        prod_id = temp_list[iterator]
        query1 = """select sold from shop_product where id ="""+str(prod_id)+""";"""
        cursor.execute(query1)
        sold = cursor.fetchall()
        sold = sold[0]
        sold = sold[0]
        
        query2 = """select view from shop_product where id ="""+str(prod_id)+""";"""
        cursor.execute(query2)
        view = cursor.fetchall()
        view = view[0]
        view = view[0]
        
        query3 = """select status from shop_product where id ="""+str(prod_id)+""";"""
        cursor.execute(query3)
        status = cursor.fetchall()
        status = status[0]
        status = status[0]
        
        query4 = """select stock from shop_product where id ="""+str(prod_id)+""";"""
        cursor.execute(query4)
        stock = cursor.fetchall()
        stock = stock[0]
        stock = stock[0]
        
        if (sold > avg_sold) and (view > avg_view):
            if (stock > min_stock) and (status is 1):
                final_products_to_be_recommended.append(prod_id)
        
    
    random.shuffle(final_products_to_be_recommended)
    
    return final_products_to_be_recommended

In [30]:
#Supposing the product description page is of the product id = 85
id_given = "96"
print("Product displayed: ")
cursor.execute("select product_id, name from shop_product_description where product_id ="+id_given+";")
displayed_product = cursor.fetchall()
print(displayed_product)
displayed_product_name = displayed_product[0]
displayed_product_name = displayed_product_name[1]
displayed_product_name
displayed_product_brand = displayed_product_name.split(',')
displayed_product_brand = displayed_product_brand[0]
print("BRAND : "+displayed_product_brand)

Product displayed: 
[(96, 'BALTRA, BVB 101 COLA BOTTLE, Vaccum Flask, Hot and Cold for 12 hours  with stainless steel, 350ml')]
BRAND : BALTRA


In [31]:
#Fetching products from the same brand

query = """select product_id from shop_product_description where name like '%"""+displayed_product_brand+"""%';"""
cursor.execute(query)
same_brand_products = cursor.fetchall()
same_brand_products = [item for t in same_brand_products for item in t]

print("\nSame brand products: ")
start_index = same_brand_products.index(int(id_given))
first_half = same_brand_products[start_index:]
second_half = same_brand_products[:start_index]
same_brand_products.clear()
same_brand_products = first_half + second_half
same_brand_products = same_brand_products[0:40]
print(same_brand_products)


Same brand products: 
[96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135]


In [32]:
#Keywords related to the product
related_keywords = []

print("Related keywords: ")
cursor.execute("select keyword from shop_product_description where product_id ="+id_given+";")
keywords = cursor.fetchall()

if keywords is None:
    keyword_related_products = []

else:
    for keyword in keywords:
        related_keywords.append(keyword[0])

    related_keywords = related_keywords[0].split(',')
    related_keywords = related_keywords[0].split('||')

    for i in range(len(related_keywords)):
        related_keywords[i] = related_keywords[i].strip()
    
    print(related_keywords)
    related_keywords = filter_related_keywords(related_keywords)
    print("\nFiltered related keywords: ")
    print(related_keywords)
    print("\nKeyword related prouducts: ")
    keyword_related_products = keyword_related_products(related_keywords)
    print(keyword_related_products)

Related keywords: 
['cola bottle', 'doubled wall bottle', 'stainless steel bottle', 'insulated flask', 'copper coated bottle', 'inner linear coated with copper bottle', 'hundredplus flat 50% discount']

Filtered related keywords: 
['cola bottle', 'doubled wall bottle', 'stainless steel bottle']

Keyword related prouducts: 
[96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109]


In [33]:
#Retrieving the categories list that the product is related to.
query = "select category_id from shop_product where id="+id_given+";"
cursor.execute(query)
categories = cursor.fetchall()

#Converting the returned query into a python list to work upon
tuple_elements = []
for tuple in categories:
    tuple_elements.append(tuple[0])
    
print(tuple_elements)
categories_list = tuple_elements[0].split(',')
print("\nRelated category id's: ")
print(categories_list)

['66,57,61,60']

Related category id's: 
['66', '57', '61', '60']


In [34]:
#Making a hash-list of all the categories and their respective id's

category_hash = {}

query = "select shop_category_id, name from shop_category_description;"
cursor.execute(query)
result = cursor.fetchall()
result

for tuple in result:
    category_hash[tuple[0]] = tuple[1]
    
category_hash

{52: 'Dining & Serving',
 53: 'Kitchen Appliances',
 54: 'Kitchen Cookware',
 55: 'Kitchen Tools',
 56: 'Kitchen Utensils',
 57: 'Storage',
 58: 'Kitchen Items',
 59: 'Kids',
 60: 'Smart & Exclusive',
 61: 'Hot Deals',
 62: 'Fork, Knives',
 63: 'Modular Kitchen Products',
 64: 'Hoods, Chimney',
 65: 'Hobs',
 66: 'Bottles, flasks, Jars',
 67: 'Casseroles, casserole Set',
 68: 'Containers, Container set',
 69: 'Boxes, Lunch Boxes, Tiffin',
 70: 'Kitchen utility Items',
 71: 'Stands, Racks, Holders',
 72: 'Kitchen Tools Set',
 73: 'Cookware Sets',
 74: 'South Indian Cookware',
 75: 'Pans, Pots, Handi, Kettle',
 76: 'Tawa',
 77: 'Pressure Cookers',
 78: 'Appliances- Hot Deals',
 79: 'Appliances- Smart & Exclusive',
 80: 'Electric Pots & Kettle',
 81: 'Dining & Serving- Hot deals',
 82: 'Dining- Smart & Exclusive',
 83: 'Tea Coffee sets',
 84: 'Casserole, Pots, Handi',
 85: 'Cutlery',
 86: 'Bakeware',
 87: 'Household',
 88: 'Gas Stove, Hobs, Cook Tops',
 89: 'Air fryer, Deep fryer',
 90: 'K

In [35]:
#Printing the related category names w.r.t to id's using the hashlist
print("Categories related to this product: ")
related_categories = []

for category in categories_list:
    related_categories.append(category_hash[int(category)])    
related_categories

Categories related to this product: 


['Bottles, flasks, Jars', 'Storage', 'Hot Deals', 'Smart & Exclusive']

In [36]:
#Retrieving the category related products
category_related_products = []

for product in categories_list:
        query = """select id from shop_product where category_id like '%"""+product+"""%';"""
        cursor.execute(query)
        category_related_products.append(cursor.fetchall())
        
category_related_products = filter_category_related_products(category_related_products)

print("\nCategory related products: ")
print(category_related_products)


Category related products: 
[92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 266, 267, 268, 318, 319, 335, 336, 337, 338, 339, 340, 349, 350, 351, 381, 382, 383, 384, 385, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538

In [37]:
#Fetching most sold products to recommend (popular)
query = """select product_id from shop_order_detail group by(product_id) order by count(product_id) DESC;"""
cursor.execute(query)
most_sold_products = cursor.fetchall()
most_sold_products = most_sold_products[0:12]
most_sold_products = [item for t in most_sold_products for item in t]
print("\nMost sold products: ")
most_sold_products


Most sold products: 


[404, 319, 787, 1144, 37, 3, 318, 407, 8, 153, 405, 443]

In [38]:
#Fetching most viewed products to recommend (popular)
query = """select id from shop_product order by shop_product.view DESC;"""
cursor.execute(query)
most_viewed_products = cursor.fetchall()
most_viewed_products = most_viewed_products[0:12]
most_viewed_products = [item for t in most_viewed_products for item in t]
print("\nMost viewed products: ")
most_viewed_products


Most viewed products: 


[646, 882, 787, 790, 1185, 1, 876, 798, 3, 149, 180, 440]

In [39]:
#Combining the most sold and most viewed products to form popular products
#Using shuffling to increase randomness
popular_products = most_sold_products + most_viewed_products
print("Before: ")
print(popular_products)
popular_products = remove_duplicates(popular_products)
print("\nAfter removing duplicates: ")
print(popular_products)
random.shuffle(popular_products)
print("\nShuffled: ")
print(popular_products)

Before: 
[404, 319, 787, 1144, 37, 3, 318, 407, 8, 153, 405, 443, 646, 882, 787, 790, 1185, 1, 876, 798, 3, 149, 180, 440]

After removing duplicates: 
[404, 319, 787, 1144, 37, 3, 318, 407, 8, 153, 405, 443, 646, 882, 790, 1185, 1, 876, 798, 149, 180, 440]

Shuffled: 
[646, 440, 790, 8, 37, 319, 443, 1, 149, 1144, 180, 876, 407, 798, 404, 153, 318, 3, 787, 882, 405, 1185]


In [40]:
products_to_recommend = recommend_products(popular_products, category_related_products, keyword_related_products)
print("\nFinal products to recommend: ")
print(products_to_recommend)


Final products to recommend: 
[801, 787, 1144, 319, 472, 3, 318, 8, 190, 468, 1, 405]
